In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
t_tokenizer = AutoTokenizer.from_pretrained("nvidia/NV-Embed-v2")

/Users/anhnguyenhoang/Desktop/vscode/python3.10/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
t_tokenizer.add_eos_token = True

In [3]:
import json, os

def load(path):
    with open(path) as f:
        data = [json.loads(line) for line in f]
    return data

dataset_names = ["MAVEN", "ACE"]
perm_ids = [0, 1, 2, 3, 4]

root="./data_incremental_2"

# for name in dataset_names:
#     root_name = root + "/" + name + "/"
#     test_path = root_name + f"{name}.test.jsonl"
#     for 


# MAVEN

In [4]:
def get_trigger(tokens, span):
    trigger = tokenizer.decode(tokens[span[0]:span[1] + 1])
    step = span[1] - span[0] + 1
    order = 0
    for i in range(len(tokens) - step + 1):
        if tokenizer.decode(tokens[i:i+step]) == trigger:
            if i == span[0]:
                break
            order += 1
    return trigger, order

def get_span(tokenized, trigger, order, prefix_length=16):
    step = len(t_tokenizer.tokenize(trigger))
    n_order = 0
    span = None
    for i in range(len(tokenized) - step + 1):
        if t_tokenizer.decoder.decode(tokenized[i:i+step]) == trigger:
            if n_order == order:
                span = [i+prefix_length, i+step-1+prefix_length]
            n_order += 1

    if not span:
        step += 1
        n_order = 0
        for i in range(len(tokenized) - step + 1):
            if t_tokenizer.decoder.decode(tokenized[i:i+step]) == trigger:
                if n_order == order:
                    span = [i+prefix_length, i+step-1+prefix_length]
                n_order += 1
    
    if not span:
        step += 1
        n_order = 0
        for i in range(len(tokenized) - step + 1):
            if t_tokenizer.decoder.decode(tokenized[i:i+step]) == trigger:
                if n_order == order:
                    span = [i+prefix_length, i+step-1+prefix_length]
                n_order += 1
    
    if not span:
        step -= 2
        order -= 1
        n_order = 0
        for i in range(len(tokenized) - step + 1):
            if t_tokenizer.decoder.decode(tokenized[i:i+step]) == trigger:
                if n_order == order:
                    span = [i+prefix_length, i+step-1+prefix_length]
                n_order += 1

    return span

def format_trigger(trigger):
    if trigger == "' s": return "'s"
    if trigger == "' re": return "'re"
    if trigger == "' ve": return "'ve"
    if trigger == "' ll": return "'ll"
    if trigger == "' d": return "'d"
    if trigger[0] == "'": return trigger[1:].strip()
    if trigger[0] == ".":
        trigger = trigger[1:].strip()
    try:
        if trigger[-1] == ".":
            trigger = trigger[:-1].strip()
    except:
        pass 
    return trigger.replace("n't", " not").strip()
    # if trigger[0]
    

In [ ]:
data = load("./data_incremental_2/MAVEN/MAVEN.test.jsonl")

prefix = t_tokenizer.encode("Classify the type of event corresponding to each token in the following sentence:")[:-1]

count = 0

with open("./data_incremental/MAVEN/MAVEN.test.jsonl", 'w') as file:
    for row in data:
        tokens = row['piece_ids']
        spans = row['span']
        labels = row['label']
        text = tokenizer.decode(tokens, skip_special_tokens=True)
        text = text.replace("n't", " not")
        new_ids = prefix + t_tokenizer.encode(text)[1:]
        
        n_spans = []
        n_tokenized = t_tokenizer.tokenize(text)
        for i, span in enumerate(spans):
            trigger, order = get_trigger(tokens, span)
            trigger = format_trigger(trigger)
            n_span = get_span(n_tokenized, trigger, order)
            if not n_span:
                del labels[i]
                print(trigger, span, order, text, tokens)
                continue
                
                # if count > 20: break
            n_spans.append(n_span)
        
        # if count > 20: break
        
        row["piece_ids"] = new_ids
        row["span"] = n_spans
        row["label"] = labels

        json_record = json.dumps(row)
        file.write(json_record + '\n')

# ACE

In [36]:
def get_trigger(tokens, span):
    trigger = tokenizer.decode(tokens[span[0]:span[1] + 1])
    step = span[1] - span[0] + 1
    order = 0
    for i in range(len(tokens) - step + 1):
        if tokenizer.decode(tokens[i:i+step]) == trigger:
            if i == span[0]:
                break
            order += 1
    return trigger, order

def get_span(tokenized, trigger, order, prefix_length=16):
    step = len(t_tokenizer.tokenize(trigger))
    n_order = 0
    span = None
    for i in range(len(tokenized) - step + 1):
        if t_tokenizer.decoder.decode(tokenized[i:i+step]) == trigger:
            if n_order == order:
                span = [i+prefix_length, i+step-1+prefix_length]
            n_order += 1

    if not span:
        step += 1
        n_order = 0
        for i in range(len(tokenized) - step + 1):
            if t_tokenizer.decoder.decode(tokenized[i:i+step]) == trigger:
                if n_order == order:
                    span = [i+prefix_length, i+step-1+prefix_length]
                n_order += 1
    
    if not span:
        step += 1
        n_order = 0
        for i in range(len(tokenized) - step + 1):
            if t_tokenizer.decoder.decode(tokenized[i:i+step]) == trigger:
                if n_order == order:
                    span = [i+prefix_length, i+step-1+prefix_length]
                n_order += 1
    
    if not span:
        step -= 2
        order -= 1
        n_order = 0
        for i in range(len(tokenized) - step + 1):
            if t_tokenizer.decoder.decode(tokenized[i:i+step]) == trigger:
                if n_order == order:
                    span = [i+prefix_length, i+step-1+prefix_length]
                n_order += 1

    return span

def format_trigger(trigger):
    if trigger == "' s": return "'s"
    if trigger == "' re": return "'re"
    if trigger == "' ve": return "'ve"
    if trigger == "' ll": return "'ll"
    if trigger == "' d": return "'d"
    if trigger[0] == "'": return trigger[1:].strip()
    if trigger[0] == ".":
        trigger = trigger[1:].strip()
    try:
        if trigger[-1] == ".":
            trigger = trigger[:-1].strip()
    except:
        pass 
    return trigger.replace("n't", " not").strip()
    # if trigger[0]
    

In [37]:
def get_valid(labels, spans):
    valid_labels = []
    for label in labels:
        if label == 0:
            break
        valid_labels.append(label)
    return valid_labels, spans[:len(valid_labels)]

def get_ignore(spans):
    ignores = []
    for span in spans:
        ignores += list(range(span[0], span[1] + 1))

    return ignores

In [38]:
data = load("./data_incremental_2/ACE/ACE.test.jsonl")

prefix = t_tokenizer.encode("Classify the type of event corresponding to each token in the following sentence:")[:-1]

count = 0

with open("./data_incremental/ACE/ACE.test.jsonl", 'w') as file:
    for row in data:
        tokens = row['piece_ids']
        spans = row['span']
        labels = row['label']

        labels, spans = get_valid(labels, spans)

        text = tokenizer.decode(tokens, skip_special_tokens=True)
        text = text.replace("n't", " not")
        new_ids = prefix + t_tokenizer.encode(text)[1:]
        
        n_spans = []
        n_tokenized = t_tokenizer.tokenize(text)
        for i, span in enumerate(spans):
            trigger, order = get_trigger(tokens, span)
            trigger = format_trigger(trigger)
            n_span = get_span(n_tokenized, trigger, order)
            if not n_span:
                del labels[i]
                print(trigger, span, order, text, tokens)
                continue
            
            # count += 1
            # if count > 10: break
            # print(trigger, span, n_span, order, text, tokens)
            # print(trigger, t_tokenizer.decode(new_ids[n_span[0]: n_span[1] + 1]))
            n_spans.append(n_span)
        
        # if count > 10: break
        ignores = get_ignore(n_spans)
        for i in range(len(prefix), len(new_ids)):
            if i not in ignores:
                n_spans.append([i,i])
                labels.append(0)

        
        row["piece_ids"] = new_ids
        row["span"] = n_spans
        row["label"] = labels

        json_record = json.dumps(row)
        file.write(json_record + '\n')

## fsed

In [46]:
for perm_id in [0, 1, 2, 3, 4]:
    for dir in os.listdir(f"./data_incremental_2/ACE/perm{perm_id}"):
        data = load(f"./data_incremental_2/ACE/perm{perm_id}/{dir}")

        prefix = t_tokenizer.encode("Classify the type of event corresponding to each token in the following sentence:")[:-1]

        with open(f"./data_incremental/ACE/perm{perm_id}/{dir}", 'w') as file:
            for task in data:
                for key in task.keys():
                    for row in task[key]:
                        tokens = row['piece_ids']
                        spans = row['span']
                        labels = row['label']

                        labels, spans = get_valid(labels, spans)

                        text = tokenizer.decode(tokens, skip_special_tokens=True)
                        text = text.replace("n't", " not")
                        new_ids = prefix + t_tokenizer.encode(text)[1:]
                        
                        n_spans = []
                        n_tokenized = t_tokenizer.tokenize(text)
                        for i, span in enumerate(spans):
                            trigger, order = get_trigger(tokens, span)
                            trigger = format_trigger(trigger)
                            n_span = get_span(n_tokenized, trigger, order)
                            if not n_span:
                                del labels[i]
                                print(trigger, span, order, text, tokens)
                                continue
                            
                            n_spans.append(n_span)
                        
                        # if count > 10: break
                        ignores = get_ignore(n_spans)
                        for i in range(len(prefix), len(new_ids)):
                            if i not in ignores:
                                n_spans.append([i,i])
                                labels.append(0)

                        
                        row["piece_ids"] = new_ids
                        row["span"] = n_spans
                        row["label"] = labels

                json_record = json.dumps(task)
                file.write(json_record + '\n')

In [42]:
data[0]['9']

[{'piece_ids': [101,
   2053,
   11640,
   1010,
   2053,
   1041,
   1011,
   5653,
   1010,
   4144,
   2024,
   1996,
   2069,
   4957,
   2000,
   2188,
   1012,
   102],
  'label': [9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  'span': [[9, 9],
   [1, 1],
   [2, 2],
   [3, 3],
   [4, 4],
   [5, 5],
   [6, 6],
   [7, 7],
   [8, 8],
   [10, 10],
   [11, 11],
   [12, 12],
   [13, 13],
   [14, 14],
   [15, 15],
   [16, 16]]},
 {'piece_ids': [101,
   2057,
   9100,
   2256,
   16495,
   2006,
   11485,
   9187,
   2006,
   2184,
   1013,
   2861,
   1013,
   2432,
   1011,
   2017,
   2342,
   2000,
   3967,
   11485,
   5030,
   2533,
   1998,
   3198,
   2065,
   2027,
   2031,
   2151,
   5030,
   1013,
   14524,
   14555,
   2800,
   1012,
   102],
  'label': [9,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0],
  'span': [[18, 18],
   

In [51]:
from utils.tools import collect_from_json
from utils.dataloader import *

streams = collect_from_json("ACE", "./data_incremental", 'stream')
# streams = [streams[l] for l in PERM[int(args.perm_id)]] # permute the stream
label2idx = {0:0}
for st in streams:
    for lb in st:
        if lb not in label2idx:
            label2idx[lb] = len(label2idx)
streams_indexed = [[label2idx[l] for l in st] for st in streams]

all_labels = []
all_labels = list(set([t for stream in streams_indexed for t in stream if t not in all_labels]))
task_idx = [i for i in range(len(streams_indexed))]
labels = all_labels.copy()



100%|██████████| 2/2 [00:00<00:00, 674.11it/s]
